In [1]:
%gui qt
%load_ext autoreload
%autoreload 2
    
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.BRepBndLib import brepbndlib
from OCC.Core.Bnd import Bnd_OBB, Bnd_Box
from OCC.Core.BRepBuilderAPI import (
    BRepBuilderAPI_Transform,
    BRepBuilderAPI_MakeEdge,
    BRepBuilderAPI_MakeWire,
    BRepBuilderAPI_MakeFace,
)
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeSphere
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.GProp import GProp_GProps
from OCC.Core.TopAbs import TopAbs_WIRE, TopAbs_EDGE, TopAbs_VERTEX
from OCC.Core.TopoDS import TopoDS_Compound, TopoDS_Wire, TopoDS_Vertex, TopoDS_Shape, TopoDS_Face
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.gp import gp_Trsf, gp_Pnt, gp_Ax3, gp_Ax1, gp_Dir, gp_Pln, gp_Vec, gp_Mat, gp_Quaternion
from OCC.Display.SimpleGui import init_display
from OCC.Core.TopLoc import TopLoc_Location
from OCC.Extend.TopologyUtils import TopologyExplorer
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB
from IPython.display import display
from IPython.display import display, HTML
from math import acos, degrees
from OCC.Extend.TopologyUtils import TopologyExplorer
from OCC.Core.Geom import Geom_CylindricalSurface


import pandas as pd
import numpy as np
import json
import math

# Start display without blocking
viewer, start_display, add_menu, add_function_to_menu = init_display()

# JSON Library path
json_path = "../data/Shape_classifier_info.json"

viewer.EraseAll()
viewer.View_Iso()
viewer.FitAll()

pyside6 backend - Qt version 6.8.3


In [2]:
# Load the STEP and extract 'solid'
def load_step(file_path: str):
    reader = STEPControl_Reader()
    status = reader.ReadFile(file_path)
    if status != 1:
        raise RuntimeError(f"Failed to read STEP file: {file_path}")
    reader.TransferRoots()
    return reader.Shape()

In [3]:
def show_origin():

# Draw global axes
    origin = gp_Pnt(0, 0, 0)
    scale = 50

    # Axis lines
    axes = [
        ((scale, 0, 0), "RED"),
        ((0, scale, 0), "GREEN"),
        ((0, 0, scale), "BLUE")
    ]

    for vec, axis_color in axes:
        end = gp_Pnt(*vec)
        edge = BRepBuilderAPI_MakeEdge(origin, end).Edge()
        viewer.DisplayShape(edge, color=axis_color, update=False)

In [4]:
def visualize(shape, display, color="CYAN", clear=False):
    """
    Displays a shape with global XYZ axes and OBB extents.
    
    Args:
        shape: The TopoDS_Shape to display.
        display: OCC display handle (e.g., from init_display()).
        color: Optional color for the shape.
        clear: If True, erase all previous shapes before displaying.
    """

    def get_occ_color(color_name):
        colors = {
            "RED": (1.0, 0.0, 0.0),
            "GREEN": (0.0, 1.0, 0.0),
            "BLUE": (0.0, 0.0, 1.0),
            "PSS_DARK_BLUE": (0.023, 0.105, 0.215),
            "PSS_LIGHT_BLUE": (0.592, 0.792, 0.921),
            "YELLOW": (1.0, 1.0, 0.0),
            "ORANGE": (1.0, 0.5, 0.0),
            "CYAN": (0.0, 1.0, 1.0),
            "MAGENTA": (1.0, 0.0, 1.0),
            "WHITE": (1.0, 1.0, 1.0),
            "GRAY": (0.5, 0.5, 0.5),
            "BLACK": (0.0, 0.0, 0.0),
        }
        r, g, b = colors.get(color_name.upper(), (1.0, 1.0, 1.0))
        return Quantity_Color(r, g, b, Quantity_TOC_RGB)
    
    # if clear:
        viewer.EraseAll()

    # Convert color name to Quantity_Color
    qcolor = get_occ_color(color)

    # Display the shape
    viewer.DisplayShape(shape, color=qcolor, update=False)

    viewer.FitAll()



In [5]:
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

def draw_obb_box(obb, viewer, color="ORANGE", update=True):
    """
    Draws the 12 edges of the OBB as lines in the viewer.
    """
    c = obb['center']
    cx, cy, cz = c.X(), c.Y(), c.Z()
    hx, hy, hz = obb['half_x'], obb['half_y'], obb['half_z']
    dx, dy, dz = obb['x_dir'], obb['y_dir'], obb['z_dir']

    # Precompute corner offsets
    offsets = [
        (+hx, +hy, +hz), (-hx, +hy, +hz), (-hx, -hy, +hz), (+hx, -hy, +hz),
        (+hx, +hy, -hz), (-hx, +hy, -hz), (-hx, -hy, -hz), (+hx, -hy, -hz)
    ]

    # Compute corners
    corners = []
    for ox, oy, oz in offsets:
        x = cx + ox * dx.X() + oy * dy.X() + oz * dz.X()
        y = cy + ox * dx.Y() + oy * dy.Y() + oz * dz.Y()
        z = cz + ox * dx.Z() + oy * dy.Z() + oz * dz.Z()
        corners.append(gp_Pnt(x, y, z))

    # Define edges by corner indices
    edges_idx = [
        (0, 1), (1, 2), (2, 3), (3, 0),  # top face
        (4, 5), (5, 6), (6, 7), (7, 4),  # bottom face
        (0, 4), (1, 5), (2, 6), (3, 7),  # vertical edges
    ]

    for i1, i2 in edges_idx:
        edge = BRepBuilderAPI_MakeEdge(corners[i1], corners[i2]).Edge()
        viewer.DisplayShape(edge, color=color, update=False)

    if update:
        viewer.FitAll()

In [6]:
# face alignemnt worst case fix, no z/x/y alignment.
# Find the largest face by area → assume it defines primary orientation.
# Align its longest edge to the Z axis.
# Find a second large face roughly orthogonal to that → use to resolve X or Y.

from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE, TopAbs_EDGE
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRep import BRep_Tool
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve, BRepAdaptor_Surface
from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Vec, gp_Ax3, gp_Trsf
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform


def robust_align_solid_from_geometry(solid, tol=1e-3):
    """
    Align a solid using its geometry:
    - Z axis is taken from the longest edge of the largest planar face
    - X/Y axes are derived from a second large face roughly perpendicular to Z
    Returns: aligned solid, transformation, and reference Ax3
    """
    # Step 1: Find the largest planar face
    explorer = TopExp_Explorer(solid, TopAbs_FACE)
    largest_face = None
    max_area = 0.0
    face_areas = []

    while explorer.More():
        face = explorer.Current()
        props = GProp_GProps()
        brepgprop.SurfaceProperties(face, props)
        area = props.Mass()
        face_areas.append((face, area))
        if area > max_area:
            max_area = area
            largest_face = face
        explorer.Next()

    if not largest_face:
        raise RuntimeError("No faces found in solid.")

    # Step 2: Get longest edge of that face
    edge_exp = TopExp_Explorer(largest_face, TopAbs_EDGE)
    longest_edge = None
    max_length = 0.0
    longest_vec = None

    while edge_exp.More():
        edge = edge_exp.Current()
        curve_handle, u1, u2 = BRep_Tool.Curve(edge)
        curve = BRepAdaptor_Curve(edge)
        p1 = curve.Value(curve.FirstParameter())
        p2 = curve.Value(curve.LastParameter())
        vec = gp_Vec(p1, p2)
        length = vec.Magnitude()
        if length > max_length:
            max_length = length
            longest_edge = edge
            longest_vec = vec
        edge_exp.Next()

    if not longest_vec:
        raise RuntimeError("No edge found on largest face.")

    z_dir = gp_Dir(longest_vec)

    # Step 3: Find a second large face with normal ≠ Z to define X
    second_face = None
    second_normal = None
    for face, area in sorted(face_areas, key=lambda x: -x[1]):
        if face.IsSame(largest_face):
            continue
        u, v = 0.5, 0.5
        surf_adapt = BRepAdaptor_Surface(face)
        normal = surf_adapt.Plane().Axis().Direction() if surf_adapt.GetType() == 0 else None
        if normal and abs(normal.Dot(z_dir)) < 0.9:  # roughly perpendicular
            second_face = face
            second_normal = normal
            break

    if not second_normal:
        raise RuntimeError("No suitable secondary face found for X/Y orientation.")

    x_dir = second_normal
    y_dir = z_dir.Crossed(x_dir)

    # Step 4: Create transformation from current CS to global
    origin = gp_Pnt(0, 0, 0)
    from_cs = gp_Ax3(origin, z_dir, x_dir)
    to_cs = gp_Ax3(origin, gp_Dir(0, 0, 1), gp_Dir(1, 0, 0))

    trsf = gp_Trsf()
    trsf.SetDisplacement(from_cs, to_cs)

    aligned = BRepBuilderAPI_Transform(solid, trsf, True).Shape()
    return aligned, trsf, to_cs, second_face

In [ ]:
from OCC.Core.gp import gp_Ax3, gp_Trsf, gp_Pnt, gp_Dir
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

def align_to_dstv_frame(solid, origin_local, dir_x, dir_y, dir_z):
    """
    Transforms a solid so that:
    - dir_x aligns to global X (beam length)
    - dir_y aligns to global Y (beam height)
    - dir_z aligns to global Z (beam width)
    - origin_local maps to global (0, 0, 0)
    """
    # Create local coordinate system from solid geometry
    local_ax3 = gp_Ax3(origin_local, dir_z, dir_x)  # Z = normal, X = length

    # Create global DSTV coordinate system: Z = up, X = length, Y = height
    global_ax3 = gp_Ax3(gp_Pnt(0, 0, 0), gp_Dir(0, 0, 1), gp_Dir(1, 0, 0))

    # Create transformation
    trsf = gp_Trsf()
    trsf.SetDisplacement(local_ax3, global_ax3)

    # Apply to shape
    transformer = BRepBuilderAPI_Transform(solid, trsf, True)
    return transformer.Shape(), trsf


In [7]:
def fingerprint_shape(shape, obb):
    """
    Compute width, height, and CSA of the cross-section.
    Attempts wire-based CSA first; falls back to building from edges if needed.
    """

    half_sizes = [obb['half_x'], obb['half_y'], obb['half_z']]
    length_idx = max(range(3), key=lambda i: half_sizes[i])
    dims = [2 * h for h in half_sizes]
    width, height = sorted([dims[i] for i in range(3) if i != length_idx], reverse=True)

    center_xyz = obb['center']
    center = gp_Pnt(center_xyz.X(), center_xyz.Y(), center_xyz.Z())
    dir_map = [obb['x_dir'], obb['y_dir'], obb['z_dir']]
    normal = dir_map[length_idx]
    plane = gp_Pln(center, normal)

    section = BRepAlgoAPI_Section(shape, plane, True)
    section.ComputePCurveOn1(True)
    section.Approximation(True)
    section.Build()

    compound = section.Shape()

    # Try wire-based approach first
    exp = TopExp_Explorer(compound, TopAbs_WIRE)
    if exp.More():
        wire = exp.Current()
    else:
        # Fallback: build wire from edges
        edge_exp = TopExp_Explorer(compound, TopAbs_EDGE)
        wire_builder = BRepBuilderAPI_MakeWire()
        edge_count = 0
        while edge_exp.More():
            edge = edge_exp.Current()
            wire_builder.Add(edge)
            edge_exp.Next()
            edge_count += 1

        if edge_count == 0:
            raise RuntimeError("No wire or edge found in section result")
        if not wire_builder.IsDone():
            raise RuntimeError("Failed to build wire from edges")

        wire = wire_builder.Wire()

    # Build face from wire and compute area
    face = BRepBuilderAPI_MakeFace(plane, wire, True).Face()
    props = GProp_GProps()
    brepgprop.SurfaceProperties(face, props)
    area = props.Mass()
    centroid = props.CentreOfMass()
    return width, height, area, face, centroid

In [8]:
def build_classification_input(obb, width, height, area):
    """
    Constructs a classification-friendly dictionary using globally aligned OBB.
    Assumes the solid is aligned with Z as the length direction.
    """
    length = 2 * obb['half_z']
    return {
        "span_web": height,
        "span_flange": width,
        "area": area,
        "length": length,
    }

In [9]:
# classify shape to JSON profile list
def classify_profile(cs, json_path, tol_dim=1.0, tol_area=0.05):
    """
    Attempts to classify a structural profile by matching its dimensions and area.
    Tries both the original and swapped width/height to handle OBB axis flips.
    """
    with open(json_path) as f:
        lib = json.load(f)
    def try_match(height, width):
        best, best_score = None, float('inf')
        for cat, ents in lib.items():
            for name, info in ents.items():
                dh = abs(height - info["height"])
                dw = abs(width  - info["width"])
                if dh > tol_dim or dw > tol_dim:
                    continue
                ea = abs(cs["area"] - info["csa"]) / info["csa"]
                if ea > tol_area:
                    continue
                el = abs(cs["length"] - info.get("length", cs["length"])) / info.get("length", cs["length"])
                score = dh + dw + ea * 100 + el * 100
                if score < best_score:
                    best_score = score
                    best = {
                        **info,
                        "Designation": name,
                        "Category": cat,
                        "Measured_height": height,
                        "Measured_width": width,
                        "Measured_area": cs["area"],
                        "Measured_length": cs["length"],
                        "Match_score": score,
                        "Profile_type": info["code_profile"]
                    }
        return best, best_score

        print(f"Best: {best}")
        print(f"Best_score: {best_score}")
    
    # Try both normal and swapped width/height
    original, score1 = try_match(cs["span_web"], cs["span_flange"])
    swapped, score2  = try_match(cs["span_flange"], cs["span_web"])

    if original and (not swapped or score1 <= score2):
        original["Swapped_dimensions"] = False
        return original
    elif swapped:
        swapped["Swapped_dimensions"] = True
        return swapped
    else:
        return None

In [10]:
def print_result_table(result, precision=2):
    """
    Pretty print classification result as a one-row DataFrame with rounded floats.
    """
    if not result:
        print("No classification result.")
        return

    df = pd.DataFrame([result])

    # Round float values
    for col in df.select_dtypes(include=['float']).columns:
        df[col] = df[col].round(precision)

    display(df.transpose().rename(columns={0: "Value"}))


In [11]:
from OCC.Core.gp import gp_Pnt, gp_Vec
from OCC.Display.SimpleGui import init_display
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

def draw_axis_arrow(viewer, origin, direction, scale=500, name="", color="RED"):
    end_point = gp_Pnt(
        origin.X() + direction.X() * scale,
        origin.Y() + direction.Y() * scale,
        origin.Z() + direction.Z() * scale
    )
    edge = BRepBuilderAPI_MakeEdge(origin, end_point).Edge()
    viewer.DisplayShape(edge, update=False, color=color)

In [12]:
def compute_axis_aligned_bounding_box(shape):
    bbox = Bnd_Box()
    brepbndlib.Add(shape, bbox)
    xmin, ymin, zmin, xmax, ymax, zmax = bbox.Get()
    
    # Half-sizes
    hx = (xmax - xmin) / 2
    hy = (ymax - ymin) / 2
    hz = (zmax - zmin) / 2
    
    center = gp_Pnt((xmin + xmax)/2, (ymin + ymax)/2, (zmin + zmax)/2)
    ax3 = gp_Ax3(center, gp_Dir(0, 0, 1), gp_Dir(1, 0, 0))
    
    return ax3, {'center': center, 'x_dir': gp_Dir(1, 0, 0), 'y_dir': gp_Dir(0, 1, 0), 'z_dir': gp_Dir(0, 0, 1),
                 'half_x': hx, 'half_y': hy, 'half_z': hz}


In [13]:
def draw_axes_from_ax3(ax3, viewer, scale=100, name=""):
    origin = ax3.Location()
    x_dir = ax3.XDirection()
    y_dir = ax3.YDirection()
    z_dir = ax3.Direction()

    draw_axis_arrow(viewer, origin, x_dir, scale=scale, name=name + "_X", color="RED")
    draw_axis_arrow(viewer, origin, y_dir, scale=scale, name=name + "_Y", color="GREEN")
    draw_axis_arrow(viewer, origin, z_dir, scale=scale, name=name + "_Z", color="BLUE")




In [14]:
if __name__ == "__main__":

    step_path = "../data/0444-1 ANGLED.step"
    # step_path = "../data/ncTest.step"
    # step_path = "../data/TestEA.step" 
    # step_path = "../data/TestEAMirror.step"
    # step_path = "../data/TestUEA.step"
    # step_path = "../data/TestUEAMirror.step"
    # step_path = "../data/TestPFC.step"
    
    # Load shape and compute OBB 
    shape_orig = load_step(step_path)
    # obb_orig, obb_geom_orig = compute_obb(shape_orig)

    # #Display origin axis
    show_origin()

    # Asume worst case and no geometry aligns, force alignment
    aligned_shape, trsf, cs, second_face = robust_align_solid_from_geometry(shape_orig)
    obb_ax3, obb_geom = compute_axis_aligned_bounding_box(aligned_shape)
    
    visualize(aligned_shape, viewer, color="PSS_LIGHT_BLUE")
    draw_obb_box(obb_geom, viewer, color="BLACK")
    
    # fingerprint shape for classification
    width, height, area, face, centroid = fingerprint_shape(aligned_shape, obb_geom)
    print(f"Width: {width:.3f}, Height: {height:.3f}, CSA: {area:.3f}")

    # classify against JSON library
    # build classifiction input
    classifier_input_data = build_classification_input(obb_geom, width, height, area)
    result = classify_profile(classifier_input_data, json_path, tol_dim=1.0, tol_area=5)
    
    if result['Swapped_dimensions']:
    ## Rotating solid 

        print("Rotating Shape")
        width, height = height, width
        result["Measured_height"] == height,
        result["Measured_width"] == width,

    print_result_table(result)

Many colors for color name BLUE, using first.
Width: 203.600, Height: 203.200, CSA: 5873.149


,Value
mass,46.1
height,203.2
width,203.6
csa,5870.0
web_thickness,7.2
flange_thickness,11.0
root_radius,10.2
toe_radius,0.0
code_profile,I
Designation,203x203x46


Below is the work on dstv origin point definition for the three different shapes.

In [15]:
# Determine channel and angle orientation based on centroid comparison
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop_SurfaceProperties

def shape_direction(face, shape_centroid):
    props = GProp_GProps()
    brepgprop.SurfaceProperties(face, props)
    face_centroid = props.CentreOfMass()

    flange_vec = gp_Vec(shape_centroid, face_centroid)
    # Determine main axis of deviation
    components = [abs(flange_vec.X()), abs(flange_vec.Y()), abs(flange_vec.Z())]
    dominant_axis = components.index(max(components))
    direction = "X" if dominant_axis == 0 else "Y" if dominant_axis == 1 else "Z"
    sign = 1 if flange_vec.Coord()[dominant_axis] > 0 else -1

    return direction, sign
    
if result['Profile_type'] != 'I':
    direction, sign = shape_direction(face, centroid)
    print(f"Profile type {result['Profile_type']} opens along {direction}, direction: {'+ve' if sign > 0 else '-ve'}")
    

In [16]:
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface

def shape_direction_from_face_normal(face):
    """
    Determines the direction and sign based on the normal of a planar face.
    Requires shape to be aligned to global axes.
    """
    surf_adapt = BRepAdaptor_Surface(face)
    
    if surf_adapt.GetType() != 0:
        raise ValueError("Face is not planar")

    normal = surf_adapt.Plane().Axis().Direction()
    
    components = [abs(normal.X()), abs(normal.Y()), abs(normal.Z())]
    dominant_axis = components.index(max(components))
    direction = "X" if dominant_axis == 0 else "Y" if dominant_axis == 1 else "Z"
    sign = 1 if normal.Coord()[dominant_axis] > 0 else -1

    return direction, sign

direction, sign = shape_direction_from_face_normal(second_face)
print(f"Direction: {direction}, Sign: {'+' if sign > 0 else '-'}")

Direction: Y, Sign: -


In [17]:

# Channel DSTV origin based on OBB limits and sign direction of flange

def pick_channel_corner_from_obb(obb, sign):
    
    center = obb['center']
    half_x = obb['half_x']
    half_y = obb['half_y']
    half_z = obb['half_z']

    cx, cy, cz = center.X(), center.Y(), center.Z()

    if sign < 0:
        # Flange opens -Y → corner at min X, max Y
        x = cx - half_x
        y = cy + half_y
    else:
        # Flange opens +Y → corner at max X, min Y
        x = cx + half_x
        y = cy - half_y

    # You can choose to place this at start or end of beam length
    z = cz - half_z  # for example: start of beam

    return gp_Pnt(x, y, z)

if result['Profile_type'] == 'U':
    # construct point
    dstv_origin_channel = pick_channel_corner_from_obb(obb_geom, sign)
    
    # Visualize
    sphere = BRepPrimAPI_MakeSphere(dstv_origin_channel, 5).Shape()
    visualize(sphere, viewer, "BLUE")

In [18]:

# for I Beam, define DSTV origin and axis for use in nc1 file.

from OCC.Core.gp import gp_Pnt, gp_Vec

def compute_dstv_origin_from_obb(obb_geom, known_width, known_height):
    """
    Returns the DSTV origin at the rear-bottom-left corner of the beam.
    Handles orientation by examining both width and height axes.
    """
    center = obb_geom['center']
    hx, hy, hz = obb_geom['half_x'], obb_geom['half_y'], obb_geom['half_z']
    x_dir, y_dir, z_dir = obb_geom['x_dir'], obb_geom['y_dir'], obb_geom['z_dir']

    axes = [
        ('x', x_dir, 2 * hx, hx),
        ('y', y_dir, 2 * hy, hy),
        ('z', z_dir, 2 * hz, hz),
    ]

    width_axis = min(axes, key=lambda a: abs(a[2] - known_width))
    axes.remove(width_axis)
    height_axis = min(axes, key=lambda a: abs(a[2] - known_height))
    axes.remove(height_axis)

    width_label, width_dir, _, hw = width_axis
    height_label, height_dir, _, hh = height_axis
    length_label, length_dir, _, hl = axes[0]

    # Rear = -Z
    rear_vec = gp_Vec(length_dir).Multiplied(hl).Reversed()
    # Bottom = -Y
    bottom_vec = gp_Vec(height_dir).Multiplied(hh).Reversed()

    # LEFT: logic based on orientation
    # if width_axis is X → +X
    # if width_axis is Y → -Y
    if width_label == 'x':
        left_vec = gp_Vec(width_dir).Multiplied(hw)
        move_desc = "+X"
    elif width_label == 'y':
        left_vec = gp_Vec(width_dir).Multiplied(hw).Reversed()
        move_desc = "-Y"
    else:
        raise ValueError("Width axis cannot be Z for I-beam")

    dstv_origin = center.Translated(left_vec).Translated(bottom_vec).Translated(rear_vec)

    print("[DSTV Origin Debug]")
    print(f"  Width axis: {width_label}, Height axis: {height_label}")
    print(f"  Left move = {move_desc}, Bottom = -{height_label}, Rear = -{length_label}")
    print(f"  Final DSTV Origin = ({dstv_origin.X():.2f}, {dstv_origin.Y():.2f}, {dstv_origin.Z():.2f})")

    return dstv_origin

def compute_raw_origin_from_obb(obb_geom):
    center = obb_geom['center']
    origin = center
    for dir_key, half_len in zip(['x_dir', 'y_dir', 'z_dir'], ['half_x', 'half_y', 'half_z']):
        vec = gp_Vec(obb_geom[dir_key]).Multiplied(obb_geom[half_len])
        origin = origin.Translated(vec.Reversed())  # go to min bound
    return origin

def assign_dstv_axes(obb_geom, known_height, known_width):
    axes = [
        ('x', obb_geom['x_dir'], 2 * obb_geom['half_x']),
        ('y', obb_geom['y_dir'], 2 * obb_geom['half_y']),
        ('z', obb_geom['z_dir'], 2 * obb_geom['half_z']),
    ]

    # Assign height and width using known dimensions
    height_axis = min(axes, key=lambda a: abs(a[2] - known_height))  # Y = height
    axes.remove(height_axis)

    width_axis = min(axes, key=lambda a: abs(a[2] - known_width))    # Z = width
    axes.remove(width_axis)

    length_axis = axes[0]  # X = remaining axis = length

    print("[DSTV Axes Assignment]")
    print(f"  X = length → {length_axis[0]}")
    print(f"  Y = height → {height_axis[0]}")
    print(f"  Z = width  → {width_axis[0]}")

    return (
        length_axis[1],  # X (beam length)
        height_axis[1],  # Y (beam height)
        width_axis[1]    # Z (beam width)
    )

def orient_axis_toward_center(axis: gp_Dir, origin: gp_Pnt, center: gp_Pnt) -> gp_Dir:
    delta = gp_Vec(origin, center)
    axis_vec = gp_Vec(axis)
    if delta.Dot(axis_vec) < 0:
        axis_vec.Reverse()
    return gp_Dir(axis_vec)

def build_right_handed_dstv_cs(origin, x_dir, y_dir):
    return gp_Ax3(origin, x_dir, y_dir)  # Z = X × Y


dstv_origin = compute_dstv_origin_from_obb(obb_geom, known_width=width, known_height=height)

origin = compute_raw_origin_from_obb(obb_geom)

sphere = BRepPrimAPI_MakeSphere(dstv_origin, 5).Shape()
visualize(sphere, viewer, "BLUE")

x_dir, y_dir, z_dir = assign_dstv_axes(obb_geom, height, width)

x_dir = orient_axis_toward_center(x_dir, dstv_origin, obb_geom['center'])
y_dir = orient_axis_toward_center(y_dir, dstv_origin, obb_geom['center'])
z_dir = orient_axis_toward_center(z_dir, dstv_origin, obb_geom['center'])

dstv_cs = build_right_handed_dstv_cs(dstv_origin, x_dir, y_dir)

draw_axes_from_ax3(dstv_cs, viewer, scale=500, name="DSTV_CS")


[DSTV Origin Debug]
  Width axis: y, Height axis: x
  Left move = -Y, Bottom = -x, Rear = -z
  Final DSTV Origin = (-101.60, -101.80, -0.00)
[DSTV Axes Assignment]
  X = length → z
  Y = height → x
  Z = width  → y
Many colors for color name BLUE, using first.


In [19]:
def pick_channel_corner_from_obb(obb, direction, sign):
    """
    Returns the DSTV origin at the back corner of the channel web,
    based on OBB and detected orientation.
    """

    cx, cy, cz = obb['center'].X(), obb['center'].Y(), obb['center'].Z()
    hx, hy, hz = obb['half_x'], obb['half_y'], obb['half_z']

    if direction == "Y":
        # Y-flange orientation → choose based on Y and X
        x = cx - hx if sign < 0 else cx + hx
        y = cy - hy if sign < 0 else cy + hy
    elif direction == "X":
        # X-flange orientation → choose based on X and Y
        x = cx - hx if sign < 0 else cx + hx
        y = cy + hy if sign < 0 else cy - hy
    else:
        raise NotImplementedError("Z-facing channels not supported")

    z = cz - hz  # start of beam

    return gp_Pnt(x, y, z)


if result['Profile_type'] == 'U':
    direction, sign = shape_direction_from_face_normal(second_face)
    dstv_origin = pick_channel_corner_from_obb(obb_geom, direction, sign)

    # Visualize
    sphere = BRepPrimAPI_MakeSphere(dstv_origin, 5).Shape()
    visualize(sphere, viewer, "RED")


In [20]:
# Get OBB leg lengths from half-extents
def get_obb_leg_lengths(obb):
    x_leg = 2 * obb['half_x']
    y_leg = 2 * obb['half_y']
    return x_leg, y_leg

# Determine longer leg from OBB
def classify_angle_from_obb(obb, tolerance_ratio=0.01):
    x_leg, y_leg = get_obb_leg_lengths(obb)
    is_equal = abs(x_leg - y_leg) < tolerance_ratio * max(x_leg, y_leg)
    longer_leg = "X" if x_leg > y_leg else "Y"
    return is_equal, longer_leg

def compute_angle_origin_at_corner(obb, toe_point, is_equal_angle, longer_leg):
    """
    Compute DSTV origin at the corner closest to the angle toe intersection point.
    """
    cx, cy, cz = obb['center'].X(), obb['center'].Y(), obb['center'].Z()
    hx, hy, hz = obb['half_x'], obb['half_y'], obb['half_z']

    # Detect which OBB quadrant the toe lies in
    x_sign, y_sign = detect_toe_quadrant(toe_point, obb['center'])

    # Use toe quadrant to pick X/Y corner
    x = cx + x_sign * hx
    y = cy + y_sign * hy

    # Compute Z origin using previous logic
    z = cz - hz if is_equal_angle or longer_leg == "Y" else cz + hz

    return gp_Pnt(x, y, z)

def detect_toe_quadrant(toe_point, obb_center):
    """
    Compare toe intersection point to OBB center to determine quadrant.
    Returns a tuple (x_sign, y_sign): -1 for min, +1 for max.
    """
    dx = toe_point.X() - obb_center.X()
    dy = toe_point.Y() - obb_center.Y()

    x_sign = -1 if dx < 0 else 1
    y_sign = -1 if dy < 0 else 1

    return x_sign, y_sign


def compute_angle_dstv_origin_from_obb(obb, direction, sign, is_equal_angle, longer_leg):
    """
    Compute DSTV origin for equal/unequal angle using OBB corner logic.
    direction: 'X' or 'Y' — primary flange direction
    sign: 1 or -1 — flange opens direction
    """
    cx, cy, cz = obb['center'].X(), obb['center'].Y(), obb['center'].Z()
    hx, hy, hz = obb['half_x'], obb['half_y'], obb['half_z']

    # Determine Z position (beam start or end)
    z = cz - hz if is_equal_angle or longer_leg == "Y" else cz + hz

    # Determine XY corner based on direction and sign
    if direction == "X":
        x = cx - hx if sign > 0 else cx + hx  # flange opens +X → toe is at -X
        y = cy - hy                            # toe always at bottom for angle
    elif direction == "Y":
        x = cx - hx                            # toe always at left for angle
        y = cy - hy if sign > 0 else cy + hy  # flange opens +Y → toe is at -Y
    else:
        raise NotImplementedError("Z-directed angles not supported")

    return gp_Pnt(x, y, z)

if result['Profile_type'] == 'L':
    is_equal_angle, longer_leg = classify_angle_from_obb(obb_geom)
    # You should already have this from fingerprint_shape
    toe_point = centroid

    dstv_origin = compute_angle_origin_at_corner(obb_geom, toe_point, is_equal_angle, longer_leg)

    # Optional: visualize
    sphere = BRepPrimAPI_MakeSphere(dstv_origin, 5).Shape()
    visualize(sphere, viewer, "RED")


In [21]:
# get holes based on origin locations

In [22]:
# from OCC.Core.gp import gp_Ax3, gp_Dir, gp_Pnt, gp_Mat, gp_Trsf, gp_Vec


# def enforce_positive_z_direction(obb_geom, z_dir):
#     center = obb_geom['center']
#     half_z = obb_geom['half_z']
    
#     # Create a reference direction vector toward positive Z
#     test_point = gp_Pnt(center.X(), center.Y(), center.Z() + half_z)
#     shape_vec = gp_Vec(center, test_point)

#     # Convert the gp_Dir to a gp_Vec to allow reversal
#     z_vec = gp_Vec(z_dir)

#     if gp_Dir(z_vec).Dot(gp_Dir(shape_vec)) < 0:
#         z_vec.Reverse()

#     return gp_Dir(z_vec)

# def determine_y_axis_from_height(obb_geom, height):
#     """
#     Picks the OBB axis that matches the classified height dimension.
#     Ensures it points in the direction of the OBB extent, not just global Y.
#     """
#     half_x = obb_geom['half_x']
#     half_y = obb_geom['half_y']
#     center = obb_geom['center']
    
#     # Create vectors from center outward in local X and Y directions
#     x_vec = gp_Vec(obb_geom['x_dir']) * half_x
#     y_vec = gp_Vec(obb_geom['y_dir']) * half_y

#     # Choose the axis that most closely matches the expected height
#     if abs(2 * half_x - height) < abs(2 * half_y - height):
#         chosen_vec = x_vec
#     else:
#         chosen_vec = y_vec

#     # Flip direction if it's pointing downward in the local OBB
#     if chosen_vec.Z() < 0:
#         chosen_vec.Reverse()

#     return gp_Dir(chosen_vec)


# def resolve_x_axis_from_obb(z_dir: gp_Dir, y_dir: gp_Dir, obb_geom: dict) -> gp_Dir:
#     """
#     Computes a consistent X direction from Z and Y axes, resolving handedness and OBB direction.

#     Returns:
#     - gp_Dir representing the correct X axis
#     """
#     x_vec = gp_Vec(z_dir.Crossed(y_dir))  # get raw X vector from cross product

#     # Check OBB for matching direction
#     x_candidates = [obb_geom['x_dir'], obb_geom['y_dir']]
#     for candidate in x_candidates:
#         if not candidate.IsEqual(y_dir, 1e-3):  # avoid using the Y direction
#             expected_x = gp_Vec(candidate.X(), candidate.Y(), candidate.Z())
#             if x_vec.Dot(expected_x) < 0:
#                 x_vec.Reverse()
#             break

#     return gp_Dir(x_vec)

# def build_strict_cs(origin: gp_Pnt, x_dir: gp_Dir, y_dir: gp_Dir, z_dir: gp_Dir) -> gp_Ax3:
#     # Ensure orthogonality and unit length (for safety)
#     assert abs(x_dir.Crossed(y_dir).Dot(z_dir)) > 1e-6, "Axes not orthogonal"
    
#     # Create gp_Ax3 with Z and X, which implicitly sets Y
#     ax3 = gp_Ax3(origin, z_dir, x_dir)
    
#     # Optionally: validate the resulting axes or embed this in a gp_Trsf if needed
#     return ax3

# def enforce_axes_point_into_shape(center: gp_Pnt, origin: gp_Pnt,
#                                    x_dir: gp_Dir, y_dir: gp_Dir, z_dir: gp_Dir):
#     """
#     Flips OCC axes (if needed) so that each one points from the origin toward the center of the solid.
#     Also ensures a right-handed system.
#     """
#     delta_vec = gp_Vec(origin, center)

#     x_vec = gp_Vec(x_dir)
#     y_vec = gp_Vec(y_dir)
#     z_vec = gp_Vec(z_dir)

#     if delta_vec.Dot(x_vec) < 0:
#         x_vec.Reverse()
#     if delta_vec.Dot(y_vec) < 0:
#         y_vec.Reverse()
#     if delta_vec.Dot(z_vec) < 0:
#         z_vec.Reverse()

#     # Ensure right-handed system: X = Z × Y
#     corrected_x = z_vec.Crossed(y_vec)
#     if corrected_x.Dot(x_vec) < 0:
#         x_vec.Reverse()

#     return gp_Dir(x_vec), gp_Dir(y_vec), gp_Dir(z_vec)


# #get axes
# z_dir = enforce_positive_z_direction(obb_geom, obb_geom['z_dir'])
# y_dir = determine_y_axis_from_height(obb_geom, height)  # uses classification height
# x_dir = resolve_x_axis_from_obb(z_dir, y_dir, obb_geom)

# # Polarity correction
# x_dir, y_dir, z_dir = enforce_axes_point_into_shape(
#     obb_geom['center'], dstv_origin, x_dir, y_dir, z_dir
# )

# dstv_cs = build_strict_cs(dstv_origin, x_dir, y_dir, z_dir)
# draw_axes_from_ax3(dstv_cs, viewer, scale=500, name="DSTV_CS")



In [23]:
# from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Vec, gp_Ax3

# def flip_axes_for_dstv_left_handed(center: gp_Pnt, dstv_origin: gp_Pnt,
#                                    x_dir: gp_Dir, y_dir: gp_Dir, z_dir: gp_Dir):
#     delta = gp_Vec(dstv_origin, center)

#     x_vec = gp_Vec(x_dir)
#     y_vec = gp_Vec(y_dir)
#     z_vec = gp_Vec(z_dir)

#     # Flip if pointing away from part
#     if delta.Dot(x_vec) < 0:
#         x_vec.Reverse()
#     if delta.Dot(y_vec) < 0:
#         y_vec.Reverse()
#     if delta.Dot(z_vec) < 0:
#         z_vec.Reverse()

#     # ✅ Enforce LEFT-HANDED system: X × Y = -Z
#     cross = x_vec.Crossed(y_vec)
#     if cross.Dot(z_vec) > 0:
#         # Not left-handed → flip X to correct it
#         x_vec.Reverse()

#     return gp_Dir(x_vec), gp_Dir(y_vec), gp_Dir(z_vec)


# def build_dstv_cs(origin: gp_Pnt, x_dir: gp_Dir, y_dir: gp_Dir, z_dir: gp_Dir) -> gp_Ax3:
#     """
#     Builds a DSTV-compatible left-handed gp_Ax3 coordinate system from given axes.
#     Validates that x × y = -z (left-handed).
#     """
#     cross = gp_Vec(x_dir).Crossed(gp_Vec(y_dir))
#     dot = cross.Dot(gp_Vec(z_dir))

#     assert dot < -1e-6, f"Axes are not left-handed (dot = {dot:.6f})"

#     # gp_Ax3 still needs right-handed input, but we pass z and x as-is.
#     # Since the axes have already been flipped to be left-handed,
#     # this won't undo the handedness — it will just build the frame.
#     return gp_Ax3(origin, z_dir, x_dir)

# def assign_beam_axes_to_obb_edges(obb_geom, width, height, dstv_origin):
#     """
#     Assigns the X, Y, Z axes based on known width and height,
#     aligning with OBB edge directions rather than guessing from axis names.
#     """
#     center = obb_geom['center']
#     obb_axes = [obb_geom['x_dir'], obb_geom['y_dir'], obb_geom['z_dir']]
#     obb_extents = [2 * obb_geom['half_x'], 2 * obb_geom['half_y'], 2 * obb_geom['half_z']]
#     labels = ['axis_0', 'axis_1', 'axis_2']

#     # Zip axes, extents and labels for easier matching
#     candidates = list(zip(labels, obb_axes, obb_extents))

#     # Match closest axis to width
#     width_axis = min(candidates, key=lambda a: abs(a[2] - width))
#     candidates.remove(width_axis)

#     # Match closest axis to height
#     height_axis = min(candidates, key=lambda a: abs(a[2] - height))
#     candidates.remove(height_axis)

#     # Remaining is length
#     length_axis = candidates[0]

#     print("Axis assignments from OBB edge alignment:")
#     print(f"  Width  → {width_axis[0]} (extent = {width_axis[2]:.2f})")
#     print(f"  Height → {height_axis[0]} (extent = {height_axis[2]:.2f})")
#     print(f"  Length → {length_axis[0]} (extent = {length_axis[2]:.2f})")

#     # Assign in strict DSTV order
#     x_dir = width_axis[1]
#     y_dir = height_axis[1]
#     z_dir = length_axis[1]

#     # Flip axes to point from origin toward center
#     x_dir, y_dir, z_dir = flip_axes_for_dstv_left_handed(center, dstv_origin, x_dir, y_dir, z_dir)

#     print("\n--- Final Axis Directions (after flipping) ---")
#     print(f"X dir (width):  ({x_dir.X():.3f}, {x_dir.Y():.3f}, {x_dir.Z():.3f})")
#     print(f"Y dir (height): ({y_dir.X():.3f}, {y_dir.Y():.3f}, {y_dir.Z():.3f})")
#     print(f"Z dir (length): ({z_dir.X():.3f}, {z_dir.Y():.3f}, {z_dir.Z():.3f})")

    
#     return x_dir, y_dir, z_dir

# x_dir, y_dir, z_dir = assign_beam_axes_to_obb_edges(
#     obb_geom,
#     width,
#     height,
#     dstv_origin,
# )

# dstv_cs = build_dstv_cs(dstv_origin, x_dir, y_dir, z_dir)
# draw_axes_from_ax3(dstv_cs, viewer, scale=500, name="DSTV_CS")


In [24]:
# Get holes from DSTV_cs, apply to dstv faces

In [25]:
# analysis and positioning of nc start point per face

DSTV_FACE_MAP = {'I':['O','U','V'], 
                 'U':['H','U','O'], 
                 'L':['H','U']}

In [26]:
# from OCC.Core.GeomAbs import GeomAbs_Cylinder
# from OCC.Core.BRep import BRep_Builder
# from OCC.Core.TopoDS import TopoDS_Compound
# from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
# from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeSphere
# from OCC.Core.TopExp import TopExp_Explorer
# from OCC.Core.TopAbs import TopAbs_FACE
# from OCC.Core.gp import gp_Dir, gp_Pnt, gp_Vec
# import pandas as pd

# DSTV_FACE_MAP = {
#     "I": ["O", "U", "V"],
#     "U": ["H", "U", "O"],
#     "L": ["H", "U"]
# }

# def make_sphere_compound(spheres):
#     builder = BRep_Builder()
#     comp = TopoDS_Compound()
#     builder.MakeCompound(comp)
#     for s in spheres:
#         builder.Add(comp, s)
#     return comp

# def get_cylinder_center(face):
#     surf_adapt = BRepAdaptor_Surface(face, True)
#     cylinder = surf_adapt.Cylinder()
#     return cylinder.Location()

# def get_cylinder_axis(face):
#     surf_adapt = BRepAdaptor_Surface(face, True)
#     cylinder = surf_adapt.Cylinder()
#     return cylinder.Axis().Direction()

# def get_cylinder_diameter(face):
#     surf_adapt = BRepAdaptor_Surface(face, True)
#     radius = surf_adapt.Cylinder().Radius()
#     return radius * 2

# def make_debug_sphere(center_pnt, radius=2.0):
#     return BRepPrimAPI_MakeSphere(center_pnt, radius).Shape()

# def is_almost_parallel(dir1: gp_Dir, dir2: gp_Dir, tol=1e-3) -> bool:
#     return abs(abs(dir1.Dot(dir2)) - 1.0) < tol

# def extract_cylindrical_faces_excluding_axis(solid, length_axis_dir, tol=1e-3):
#     cylindrical_faces = []
#     exp = TopExp_Explorer(solid, TopAbs_FACE)
#     while exp.More():
#         face = exp.Current()
#         surf_adapt = BRepAdaptor_Surface(face, True)
#         if surf_adapt.GetType() == GeomAbs_Cylinder:
#             axis_dir = surf_adapt.Cylinder().Axis().Direction()
#             if not is_almost_parallel(axis_dir, length_axis_dir, tol):
#                 cylindrical_faces.append(face)
#         exp.Next()
#     return cylindrical_faces

# # def classify_dstv_face(hole_axis: gp_Dir, DSTV_cs, profile_type: str) -> str:
# #     face_map = DSTV_FACE_MAP.get(profile_type, [])
    
# #     # Axis definitions based on swapped DSTV CS
# #     axis_width  = DSTV_cs.XDirection()   # Flange direction (old Z)
# #     axis_height = DSTV_cs.YDirection()   # Web direction
# #     axis_length = DSTV_cs.Direction()    # Length (feed)

# #     dot_width  = hole_axis.Dot(axis_width)
# #     dot_height = hole_axis.Dot(axis_height)
# #     dot_length = hole_axis.Dot(axis_length)

# #     flange_dir = DSTV_cs.Direction()
# #     dot_flange = hole_axis.Dot(flange_dir)

# #     # Use dot_width for O/U, dot_height for V
# #     if dot_flange < -0.9 and "O" in face_map:
# #         return "O"
# #     elif dot_flange > 0.9 and "U" in face_map:
# #         return "U"
# #     elif abs(dot_height) > 0.9 and "V" in face_map:
# #         return "V"
# #     elif "H" in face_map:
# #         return "H"
# #     return "UNKNOWN"

# def classify_dstv_face(hole_axis: gp_Dir, DSTV_cs, profile_type: str, center_Y: float, mid_Y: float) -> str:
#     face_map = DSTV_FACE_MAP.get(profile_type, [])
    
#     axis_width  = DSTV_cs.XDirection()
#     axis_height = DSTV_cs.YDirection()
#     axis_length = DSTV_cs.Direction()

#     dot_width  = hole_axis.Dot(axis_width)
#     dot_height = hole_axis.Dot(axis_height)

#     # Debug print for diagnostics
#     print(f"    Dot Width: {dot_width:.3f}, Dot Height: {dot_height:.3f}, Y = {center_Y:.2f}")

#     # Flange holes (code O or U) based on width direction and Y position
#     if abs(dot_width) > 0.9:
#         if center_Y < mid_Y and "O" in face_map:
#             return "O"
#         elif center_Y >= mid_Y and "U" in face_map:
#             return "U"

#     # Web holes (code V)
#     elif abs(dot_height) > 0.9 and "V" in face_map:
#         return "V"

#     # H or fallback
#     elif "H" in face_map:
#         return "H"

#     return "UNKNOWN"



# def extract_holes_relative_to_dstv_cs(
#     solid,
#     DSTV_cs,
#     profile_type,
#     face_center_func,
#     face_axis_func,
#     face_diameter_func,
#     tol=1e-3
# ):
#     def to_dstv_coords(pnt_global: gp_Pnt, DSTV_cs) -> tuple:
#         origin = DSTV_cs.Location()
#         vec = gp_Vec(origin, pnt_global)
#         x = vec.Dot(gp_Vec(DSTV_cs.XDirection()))
#         y = vec.Dot(gp_Vec(DSTV_cs.YDirection()))
#         z = vec.Dot(gp_Vec(DSTV_cs.Direction()))
#         return x, y, z

#     length_axis = DSTV_cs.Direction()
#     cylindrical_faces = extract_cylindrical_faces_excluding_axis(solid, length_axis, tol)

#     hole_data = []
#     color_map = {"O": "red", "U": "green", "V": "blue", "H": "orange", "UNKNOWN": "gray"}

#     # Precompute all centers and Y values
#     all_centers = [face_center_func(face) for face in extract_cylindrical_faces_excluding_axis(solid, DSTV_cs.Direction(), tol)]
#     all_y_vals = [gp_Vec(DSTV_cs.Location(), p).Dot(gp_Vec(DSTV_cs.YDirection())) for p in all_centers]
#     mid_Y = sum(all_y_vals) / len(all_y_vals) if all_y_vals else 0.0

    
#     for i, face in enumerate(cylindrical_faces):
#         center = face_center_func(face)
#         axis = face_axis_func(face)
#         diameter = face_diameter_func(face)

#         # Transform to DSTV CS
#         x, y, z = to_dstv_coords(center, DSTV_cs)
        
#         # DEBUG: check Y height to distinguish U vs O
#         print(f"Hole {i+1}: axis={axis}, center Y in DSTV CS = {round(y,2)}")
        
#         # code = classify_dstv_face(axis, DSTV_cs, profile_type)
#         code = classify_dstv_face(axis, DSTV_cs, profile_type, y, mid_Y)


#         if code == "UNKNOWN":
#             print(f"⚠️ Hole {i+1} could not be classified! Axis: {axis}")
      
#         sphere = make_debug_sphere(center, radius=5.0)
#         viewer.DisplayShape(sphere, update=False, color=color_map.get(code, "black"))

#         print(f"Hole {i+1}: axis={axis}, code={code}")

#         x, y, z = to_dstv_coords(center, DSTV_cs)

#         if code in ["O", "U"]:
#             x_pos, y_pos = y,z
#         elif code in ["V", "H"]:
#             x_pos, y_pos = x, z
#         else:
#             x_pos = y_pos = None
        
#         hole_data.append({
#             "Hole #": i + 1,
#             "Code": code,
#             "Diameter (mm)": round(diameter, 2),
#             "X (mm)": round(x, 2),
#             "Y (mm)": round(y, 2),
#             "Z (mm)": round(z, 2),
#         })

#     return pd.DataFrame(hole_data)


In [27]:
# from OCC.Core.gp import gp_Dir
# from math import acos, degrees

# df_holes = extract_holes_relative_to_dstv_cs(
#     solid=aligned_shape,
#     DSTV_cs=dstv_cs,
#     profile_type=result['Profile_type'],
#     face_center_func=get_cylinder_center,
#     face_axis_func=get_cylinder_axis,
#     face_diameter_func=get_cylinder_diameter
# )

# print(df_holes)

In [28]:
# Create V/U/O planes

In [29]:
from OCC.Core.gp import gp_Pln, gp_Ax3

from OCC.Core.gp import gp_Pln, gp_Ax3, gp_Pnt, gp_Dir, gp_Vec
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeFace

def make_plane_face_cornered(pln: gp_Pln, dir_u: gp_Dir, dir_v: gp_Dir, half_u: float, half_v: float):
    # Start at center and shift to corner
    shift_vec = gp_Vec(dir_u).Multiplied(-half_u).Added(gp_Vec(dir_v).Multiplied(-half_v))
    origin_shifted = pln.Location().Translated(shift_vec)
    pln_moved = gp_Pln(gp_Ax3(origin_shifted, pln.Axis().Direction(), pln.XAxis().Direction()))
    return BRepBuilderAPI_MakeFace(pln_moved, 0, 2 * half_u, 0, 2 * half_v).Face()


def create_dstv_face_planes(dstv_cs: gp_Ax3, half_x: float, half_y: float, half_z: float):
    origin = dstv_cs.Location()
    axis_x = dstv_cs.XDirection()   # Flange width direction
    axis_y = dstv_cs.YDirection()   # Web height direction
    axis_z = dstv_cs.Direction()    # Length direction

    # Top flange (U)
    pln_U = gp_Pln(gp_Ax3(origin, axis_x, axis_y))
    face_U = make_plane_face_cornered(pln_U, axis_y, axis_z, half_y, half_z)

    # Bottom flange (O) – offset in -Y
    origin_O = origin.Translated(gp_Vec(axis_y).Multiplied(-2 * half_y))
    pln_O = gp_Pln(gp_Ax3(origin_O, axis_x, axis_y))
    face_O = make_plane_face_cornered(pln_O, axis_y, axis_z, half_y, half_z)

    # Web (V) – center, faces XZ
    pln_V = gp_Pln(gp_Ax3(origin, axis_y, axis_x))
    face_V = make_plane_face_cornered(pln_V, axis_x, axis_z, half_x, half_z)

    return {"U": face_U, "O": face_O, "V": face_V}


half_x = obb_geom['half_x']
half_y = obb_geom['half_y']
half_z = obb_geom['half_z']

planes = create_dstv_face_planes(dstv_cs, half_x, half_y, half_z)

viewer.DisplayShape(planes["U"], color="green", update=False)
viewer.DisplayShape(planes["O"], color="red", update=False)
viewer.DisplayShape(planes["V"], color="blue", update=True)



Many colors for color name blue, using first.


[<class 'AIS_Shape'>]